In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [2]:
import numpy as np
np.random.seed(5)

from keras.layers import Input, Dense, SimpleRNN
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import backend as K

In [3]:
# 1. LECTURA DEL SET DE DATOS
# ===========================================================
nombres = open('/content/gdrive/MyDrive/Colab Notebooks/SIS_421/Primer_Parcial/3er_Ejercicio/DataPalabrasInfantiles.txt','r').read()
nombres = nombres.lower()
#print(nombres)
# Crear diccionario (listado de caracteres que no se repiten)
alfabeto = list(set(nombres))
#print(alfabeto)
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (tam_datos, tam_alfabeto))

# Conversión de caracteres a índices y viceversa
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))}
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))}
print(car_a_ind)
print(ind_a_car)

En total hay 10416 caracteres, y el diccionario tiene un tamaño de 34 caracteres.
{'\n': 0, ' ': 1, ',': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'x': 25, 'y': 26, 'z': 27, 'á': 28, 'é': 29, 'í': 30, 'ñ': 31, 'ó': 32, 'ú': 33}
{0: '\n', 1: ' ', 2: ',', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'x', 26: 'y', 27: 'z', 28: 'á', 29: 'é', 30: 'í', 31: 'ñ', 32: 'ó', 33: 'ú'}


In [4]:
# 2. MODELO
# ===========================================================
n_a = 25    # Número de unidades en la capa oculta
entrada  = Input(shape=(None,tam_alfabeto))
a0 = Input(shape=(n_a,))

celda_recurrente = SimpleRNN(n_a, activation='tanh', return_state = True)
capa_salida = Dense(tam_alfabeto, activation='softmax')

salida = []
hs, _ = celda_recurrente(entrada, initial_state=a0)
salida.append(capa_salida(hs))
modelo = Model([entrada,a0],salida)
modelo.summary()

opt = SGD(lr=0.0005)
modelo.compile(optimizer=opt, loss='categorical_crossentropy')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 34)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
simple_rnn (SimpleRNN)          [(None, 25), (None,  1500        input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 34)           884         simple_rnn[0][0]             

In [6]:
# 3. EJEMPLOS DE ENTRENAMIENTO
# ===========================================================

# Crear lista con ejemplos de entrenamiento y mezclarla aleatoriamente
with open("/content/gdrive/MyDrive/Colab Notebooks/SIS_421/Primer_Parcial/3er_Ejercicio/DataPalabrasInfantiles.txt") as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

# Crear ejemplos de entrenamiento usando un generador
def train_generator():
    while True:
        # Tomar un ejemplo aleatorio
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]

        # Convertir el ejemplo a representación numérica
        X = [None] + [car_a_ind[c] for c in ejemplo]

        # Crear "Y", resultado de desplazar "X" un caracter a la derecha
        Y = X[1:] + [car_a_ind['\n']]

        # Representar "X" y "Y" en formato one-hot
        x = np.zeros((len(X),1,tam_alfabeto))
        onehot = to_categorical(X[1:],tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
        x[1:,:,:] = onehot
        y = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)

        # Activación inicial (matriz de ceros)
        a = np.zeros((len(X), n_a))

        yield [x, a], y
        

In [7]:
# 4. ENTRENAMIENTO
# ===========================================================
BATCH_SIZE = 80			# Número de ejemplos de entrenamiento a usar en cada iteración
NITS = 10000			# Número de iteraciones

for j in range(NITS):
    historia = modelo.fit(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)

    # Imprimir evolución del entrenamiento cada 1000 iteraciones
    if j%1000 == 0:
        print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')


Iteración: 0, Error: 3.545608


Iteración: 1000, Error: 2.717282


Iteración: 2000, Error: 2.520806


Iteración: 3000, Error: 2.483682


Iteración: 4000, Error: 2.485423


Iteración: 5000, Error: 2.467588


Iteración: 6000, Error: 2.366195


Iteración: 7000, Error: 2.422914


Iteración: 8000, Error: 2.329338


Iteración: 9000, Error: 2.325439



In [9]:
# 5. GENERACIÓN DE NOMBRES USANDO EL MODELO ENTRENADO
# ===========================================================
def generar_nombre(modelo,car_a_num,tam_alfabeto,n_a):
    # Inicializar x y a con ceros
    x = np.zeros((1,1,tam_alfabeto,))
    a = np.zeros((1, n_a))

    # Nombre generado y caracter de fin de linea
    nombre_generado = ''
    fin_linea = '\n'
    car = -1

    # Iterar sobre el modelo y generar predicción hasta tanto no se alcance
    # "fin_linea" o el nombre generado llegue a los 50 caracteres
    contador = 0
    while (car != fin_linea and contador !=100 ):
          # Generar predicción usando la celda RNN
          a, _ = celda_recurrente(K.constant(x), initial_state=K.constant(a))
          y = capa_salida(a)
          prediccion = K.eval(y)

          # Escoger aleatoriamente un elemento de la predicción (el elemento con
          # con probabilidad más alta tendrá más opciones de ser seleccionado)
          ix = np.random.choice(list(range(tam_alfabeto)),p=prediccion.ravel())

          # Convertir el elemento seleccionado a caracter y añadirlo al nombre generado
          car = ind_a_car[ix]
          nombre_generado += car

          # Crear x_(t+1) = y_t, y a_t = a_(t-1)
          x = to_categorical(ix,tam_alfabeto).reshape(1,1,tam_alfabeto)
          a = K.eval(a)

          # Actualizar contador y continuar
          contador += 1

          # Agregar fin de línea al nombre generado en caso de tener más de 50 caracteres
          if (contador == 100):
            nombre_generado += '\n'

    print(nombre_generado)

# Generar 100 ejemplos de nombres generados por el modelo ya entrenado
for i in range(100):
    generar_nombre(modelo,car_a_ind,tam_alfabeto,n_a)

e po

ma

úzadove

date

,la

narin

pave

nta

maro

fío

arerirararriolerolate

pe

eraralarara

maraya

s

gorvajtherara

carajiteir

í



r



z

árarerrero

rindr

ame

r

jo

ninura

carama

varenoremo

vedaro

a

aliso

sarano



pbrrcur

eo

aseviro

esorr

alo

noreripe

muzanarmlalinare

muíarrorarereve

a

l

a

erercaronarete

jo

fuciduto

ada

ónaresato

arore

efilaro

pelídova

ceyido

zeva

sarido

sariicarañe

teia

isare

póna

e

úámidir

y

tonipo

o

elorenirara

arosiro

clua

ediacoa

caroyuresererr

siolrenbrrrrisoro

nerara

íro

ema

ma

ala

ve

o

íavo

araridara

o

tr



alixovo

nte

yorororireda

tl

asopa

o

n

íarararicuravurisesarisinaju

elo

narivaro

o

o

ara

aro

mio

pedenoraca

dabiópiaro

vesfrí

e

aro

pata

peja

e

elo

lejo

íara

iabuirusica

maro

ara

aga

usoya

raór

cironariroápa

aliace



o

uenarlóá

o

aleúparediroso

ararava

e

o

alido

eco

lugrararare

uciareva

pe

ócoridí

diie

s

daya

ma

arironeqa

d

uiasuearrarop